In [1]:
import tkinter as tk
import ttkbootstrap 
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


#table_time=[0.0,0.0,0.0]
table_time=[0.0,0.0]
table_distance=[0.0,0.0,0.0,0.0]
tnum=[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
tdis=[0.0,0.0,0.0,0.0]

def timetext(s):
    return '  '+str(s)+'s'

##GUI function
def show_page(page):
    page1.pack_forget()
    page2.pack_forget()
    page.pack()
    return

def page1_apply():
    selected_partDC=comboDC.get()
    selected_partSC=comboSC.get()
    
    inputValueL=float(entryPL.get()) #L
    inputValueK=float(entryCV.get())  #K
    inputValueN=float(entryNS.get())  #N
    inputValueA=float(entryTA.get())#A

    #输入校验 不可小于等于0
    if inputValueL<=0 or inputValueL<=0 or inputValueN<=0 or inputValueA<=0:
        print('The inputs can\'t less than or equal to 0')
        return
    
    #0:h  1:c  2:Θmax  3Θmin 
    num=[
        [0.812,0.659,1,0.082],   #Head
        [0.709,0.712,1,0.099],   #Torso
        [0.683,0.453,1,0.054]]   #Knee
    
    i=0;#set type of method
    if selected_partSC == "head":
        i=0
    elif selected_partSC == "torso":
        i=1
    elif selected_partSC == "knee":
        i=2
    if selected_partDC=="head":
        f=0.01
        g=0.707
    elif selected_partDC=="torso":
        f=0.012
        g=0.865
    h=num[i][0]
    c=num[i][1]
    thetaMax=num[i][2]
    thetaMin=num[i][3]
    L=inputValueL
    K=inputValueK
    N=inputValueN
    A=inputValueA
    
    
    ##——————————————————————————————————表单的公式
    table_time=[0.0,0.0]        #表格中3个值，0:MT, 1:TT, 2:ST
    
    #table_time[0]=(L/(h*K*thetaMax))+(N*c*math.log((thetaMax*h)/thetaMin))
    #table_time[1]=f*A+g
    #table_time[2]=table_time[0]+table_time[1]
    
    #表格中2个值，0:TT, 1:ST, MT
    MT=(L/(h*K*thetaMax))+(N*c*math.log((thetaMax*h)/thetaMin))
    table_time[0]=f*A+g
    table_time[1]=table_time[0]+MT
    
    table_time = [round(value, 3) for value in table_time]
    ##——————————————————————————————————end
    show_page(page2)

    #col = ['Movement Time', 'Turning Time', 'Sum Time']
    col = ['Turning Time', 'Total Locomotion Time']
    vals=[table_time]
    table_ax = ax_table.table(cellText=vals, colLabels=col, loc='center', cellLoc='center', rowLoc='center')
    table_ax.auto_set_font_size(False)
    table_ax.set_fontsize(10)

    cell_width = 0.4
    cell_height = 0.3
    for key, cell in table_ax._cells.items():
        cell.set_text_props(weight='bold')  # 设置粗体
        cell.set_width(cell_width)
        cell.set_height(cell_height)
    canvas_plot.draw()
    return

def page2_apply():
    show_page(page1)#

def page1_clear():
    #row 1
    comboDC.set('')
    entryNS.delete(0,tk.END)
    entryCV.delete(0,tk.END)
    #row 2
    comboSC.set('')
    entryPL.delete(0,tk.END)
    entryTA.delete(0,tk.END)
    
def page2_clear():
    show_page(page1)

def page2_back():
    show_page(page1)

win = tk.Tk()
win.wm_title('Time Prediction Model')
style=ttkbootstrap.Style(theme='cosmo')
window=style.master
#win.configure(bg='#F5F5F5')
win.configure(bg='#B0E2FF')
bold_font=tk.font.Font(family="Helvetica", size=10, weight="bold")

##win.geometry("700x300")
#—————————————————————————page 1———————————————————————————————

page1 = tk.Frame(win,width=630, height=300)
page1.pack(expand=True)

# method 2 
#——row one.  notes: fixed distance:200x 30y
#text
labelDC=tk.Label(page1,text="Body Part for Direction Control",font=bold_font)
labelDC.place(x=20,y=60)
labelNS=tk.Label(page1,text="Number of Segments",font=bold_font)
labelNS.place(x=230, y=60)
labelCV = tk.Label(page1, text="Coefficient Value:",font=bold_font)
labelCV.place(x=430, y=60)
#input
selectionDC = tk.StringVar()
options = ["head", "torso"]
comboDC = tk.ttk.Combobox(page1, textvariable=selectionDC, values=options, width=18)
comboDC.place(x=20, y=90, width=150,height=25)
entryNS = tk.Entry(page1)
entryNS.place(x=230, y=90, width=150,height=25)
entryCV = tk.Entry(page1)
entryCV.place(x=430, y=90,width=150,height=25)

#——row two
labelSC=tk.Label(page1,text="Body Part for Speed Control",font=bold_font)
labelSC.place(x=20,y=150)
labelPL=tk.Label(page1,text="Path Length",font=bold_font)
labelPL.place(x=230, y=150)
labelTA = tk.Label(page1, text="Turning Angle",font=bold_font)
labelTA.place(x=430, y=150)
#input
selectionSC = tk.StringVar()
options = ["head", "torso","knee"]
comboSC = tk.ttk.Combobox(page1, textvariable=selectionSC, values=options, width=18)
comboSC.place(x=20, y=180, width=150,height=25)
entryPL = tk.Entry(page1)
entryPL.place(x=230, y=180, width=150,height=25)
entryTA = tk.Entry(page1)
entryTA.place(x=430, y=180,width=150,height=25)
#button
buttonApply = tk.Button(page1, text="apply", width=8,command=page1_apply)
buttonApply.place(x=230, y=240)
buttonClear = tk.Button(page1, text="clear", width=8,command = page1_clear)
buttonClear.place(x=330, y=240)
#———————————————————————END page 1———————————————————————————————
#—————————————————————————page 2———————————————————————————————

page2 = tk.Frame(win,width=630, height=300)
page2.pack(expand=True)
page2.pack_propagate(False)  # 禁用自动调整大小

frame_top = tk.Frame(page2,width=630,height=240)
frame_bottom = tk.Frame(page2,width=630,height=60)
frame_top.pack(side='top', fill='both', expand=True)
frame_bottom.pack(side='bottom', fill='both', expand=True)

frame_bottom.grid_rowconfigure(0, weight=1)
frame_bottom.grid_columnconfigure(0, weight=1)

frame_top.pack_propagate(False)  # 禁用自动调整大小
frame_bottom.pack_propagate(False)  # 禁用自动调整大小

# TOP frame - Table
#col = ['Movement Time', 'Turning Time', 'Sum Time']
col = ['Turning Time', 'Total Locomotion Time']
vals = [table_time]

# Create a fixed-size Figure for the table
fig = Figure()
fig.subplots_adjust(hspace=0)  # adjust the padding of two subsplots
ax_table = fig.add_subplot()
ax_table.axis('off')  # 不显示图形坐标轴
table_ax = ax_table.table(cellText=vals, colLabels=col, loc='center', cellLoc='center', rowLoc='center')
table_ax.auto_set_font_size(False)
table_ax.set_fontsize(10)

# 为整个表格的所有单元格设置粗体
for key, cell in table_ax.get_celld().items():
    cell.set_text_props(weight='bold')  # 设置粗体

cell_width = 0.4
cell_height = 0.2
for key, cell in table_ax._cells.items():
    cell.set_width(cell_width)
    cell.set_height(cell_height)
    
canvas_plot = FigureCanvasTkAgg(fig, master=frame_top)
canvas_widget_plot = canvas_plot.get_tk_widget()
canvas_widget_plot.pack(expand=True,fill="both")

back = tk.Button(frame_bottom, text="Back", width=8,  command=page2_back)
#back.pack()
back.grid(row=0, column=0, sticky='se', padx=20, pady=20)


#———————————————————————END page 2———————————————————————————————

page2.pack_forget()
page1.pack()
win.mainloop()
